In [6]:
import numpy as np
import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

from torch.nn import Linear, Conv2d, BatchNorm2d, MaxPool2d, Dropout2d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

In [2]:
num_classes = 2

# Image is grayscale, so number of channels is one
channels = 1

# 256x256 pixels each image
height = 256
width = 256


In [3]:
#  Number of filters in each convolutional layer
num_filters_conv1 = 8 
kernel_size_conv1 = 4 # [height, width]
stride_conv1 = 2 # [stride_height, stride_width]

# Number of Neurons in the last fully connected layer
num_l1 = 1000
padding_conv1 = 0
   
#  Source: CNN introduction jupyter notebook ROBT 407 Course, Nazarbayev University
#  function to calculate the dimension of the image after one convolutional layer
def compute_conv_dim(dim_size):
    return int((dim_size - kernel_size_conv1 + 2 * padding_conv1) / stride_conv1 + 1)

In [4]:
# define network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
#         out_dim = (input_dim - filter_dim + 2*padding) / stride + 1
        self.conv_1 = Conv2d(in_channels=channels,
                            out_channels=num_filters_conv1,
                            kernel_size=kernel_size_conv1,
                            stride=(2,2))
        self.pool1 = MaxPool2d(kernel_size=2, stride=2)
        self.conv_out_height = compute_conv_dim(height)
        self.conv_out_width = compute_conv_dim(width)        
        # add dropout to network
        self.dropout = Dropout2d(p=0.3)
        
        self.conv_2 = Conv2d(in_channels=num_filters_conv1,
                            out_channels=num_filters_conv1*2,
                            kernel_size=kernel_size_conv1,
                            stride=stride_conv1)
        
        self.pool2 = MaxPool2d(kernel_size=2, stride=2)
        
#         self.conv_3 = Conv2d(in_channels=num_filters_conv1*2,
#                             out_channels=num_filters_conv1*2,
#                             kernel_size=kernel_size_conv1,
#                             stride=stride_conv1)
        
#         self.pool3 = MaxPool2d(kernel_size=2, stride=2)
        
        self.l1_in_features = num_filters_conv1*2 *225
        
#         self.l1_in_features = num_filters_conv1*2 *9

#         self.l1_in_features = 2883 # 3844
        
        
        
        self.l_1 = Linear(in_features=self.l1_in_features,
                          out_features=num_l1,
                          bias=True)        
        self.l_out = Linear(in_features=num_l1, 
                            out_features=1,
                            bias=False)    
    def forward(self, x): # x.size() = [batch, channel, height, width]
        x = x.view(-1, 1, 256,256)
#         x = self.pool2(x)
        x = relu(self.conv_1(x))
#         print(x.shape)
        x = self.pool1(x)
#         print(x.shape)
        x = self.pool2(relu(self.conv_2(x)))
  
#         x = self.pool3(relu(self.conv_3(x)))
#         print(x.shape)
#         print(x.shape)
        x = x.view(-1, self.l1_in_features)
#         print(x.shape)
        x = self.dropout(relu(self.l_1(x)))
#         print(x.shape)
#         x = softmax(self.l_out(x), dim=1)
        x = F.sigmoid(self.l_out(x))
#         print(x.shape)
        return x
net = Net()
print(net)

Net(
  (conv_1): Conv2d(1, 8, kernel_size=(4, 4), stride=(2, 2))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout2d(p=0.3)
  (conv_2): Conv2d(8, 16, kernel_size=(4, 4), stride=(2, 2))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (l_1): Linear(in_features=3600, out_features=1000, bias=True)
  (l_out): Linear(in_features=1000, out_features=1, bias=False)
)


In [7]:
x_train = np.load('../data/x_train.npy')
x_train = x_train.tolist()
y_train = np.load('../data/y_train.npy')
y_train = y_train.tolist()

In [8]:
#  Randomly split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

NameError: name 'train_test_split' is not defined

In [ ]:
X_train = X_train.reshape(-1, 256, 256)
X_test = X_test.reshape(-1, 256, 256)
cv = 20
learning_rate = 0.001
num_epochs = 6
print_every = 3

from sklearn.model_selection import KFold

names = ["Net"]
nets = [Net()]
c = 1
for i in range(len(nets)):
  print("{}:".format(names[i]))
  net = nets[i]
  net.cuda()

  # lists to record loss values for plotting
  train_list = []
  val_list = []

  # Binarry Cross Entropy loss
  criterion = nn.BCELoss()

  # Opimizing with Adam, choose alpha as 0.001 for stability
  optimizer = optim.Adam(net.parameters(), lr=learning_rate)


  for j in range(num_epochs + 1):
  #   Here KFold used for batching with the test, not for the cross validation
      kf = KFold(n_splits=cv)

      running_loss = 0.0
      for train_index, test_index in kf.split(X_train):

  #     Convert current train set to pytorch float variable 
        x = Variable(torch.from_numpy(np.array(X_train[test_index]))).float()
        y = Variable(torch.from_numpy(np.array(y_train[test_index]))).float()

#         print(x.shape)
        optimizer.zero_grad()

  #     calculate loss of the train data
        outputs = net(x.cuda())
        loss = criterion(outputs, y.cuda())

  #     calculate the loss for the validation set
        x_ = Variable(torch.from_numpy(X_test))
        x_ = x_.float()
        y_ = Variable(torch.from_numpy(y_test))
        y_ = y_.float()
        outputs_ = net(x_.cuda())
        val_loss = criterion(outputs_, y_.cuda())

  #     backward step, propagate the loss
        loss.backward()

  #     update weights 
        optimizer.step()

  #     update current super batch level loss
        running_loss += loss.item()

  #     fix current train and validation loss
        train_list.append(loss.item())
        val_list.append(val_loss.item())


      if j%print_every == 0: 
        print("Epoch: {}, train loss: {}, validation_loss: {}".format( j, running_loss/cv, val_list[-1]))

  print('Finished Training')


  import matplotlib.pyplot as plt
  plt.title(names[i])
  plt.plot(train_list, label="Train loss")
  plt.plot(val_list, label="Validation loss")
  plt.legend()
  plt.savefig('opimization.png')
  plt.show()

In [ ]:
# plt.title(names[i])
# plt.plot(train_list, label="Train loss")
# plt.plot(val_list, label="Validation loss")
# plt.legend()
# plt.savefig('opimization.png', dpi=400)
# plt.show()

In [ ]:
test_results = []
for i in range(len(nets)):
    print("{}:".format(names[i]))
    net = nets[i]


    x_test = Variable(torch.from_numpy(X_test))
    x_test = x_test.float()
    outputs = net(x_test.cuda() )
    outputs = outputs.cpu().detach().numpy()
    print('Accuracy test: {}'.format(accuracy_score(y_test, np.round(outputs ))))
    print('AUC test: {}'.format(roc_auc_score(y_test, outputs)))
    print('F1 test: {}'.format(f1_score(y_test, outputs.round()  )))

    test_results.append(roc_auc_score(y_test, outputs))


    x_train = Variable(torch.from_numpy(X_train))
    x_train = x_train.float()
    outputs = net(x_train.cuda() )
    outputs = outputs.cpu().detach().numpy()
    print('Accuracy train: {}'.format(accuracy_score(y_train, np.round(outputs ))))
    print('AUC train:{}'.format(roc_auc_score(y_train, outputs)))
    print('F1 test: {}'.format(f1_score(y_train,  outputs.round()  )))